Fetch Data (Statusmeldung, unformatierte Ausgabe)

In [ ]:
import requests

url = "https://v3.football.api-sports.io/leagues"

headers = {
    "x-apisports-key": "261d42af5fbae54096924d7e009a10b1"  
}

response = requests.get(url, headers=headers)
    
# Zeigt den Statuscode (optional)
print("Status:", response.status_code)

# Gibt die Daten als Text (JSON) aus
print(response.text)


Fetch data (lesbarer output)

In [ ]:
import requests
import json

url = "https://v3.football.api-sports.io/leagues"
headers = {"x-apisports-key": "261d42af5fbae54096924d7e009a10b1"}

response = requests.get(url, headers=headers)

# JSON hübsch formatiert drucken
data = response.json()
print(json.dumps(data, indent=2))


Terminal command (ähnliche Fetch wie oben)

In [2]:
'''
curl -X GET "https://v3.football.api-sports.io/leagues" -H "x-apisports-key: 261d42af5fbae54096924d7e009a10b1
'''

'\ncurl -X GET "https://v3.football.api-sports.io/leagues" -H "x-apisports-key: 261d42af5fbae54096924d7e009a10b1\n'

# Jan am usprobiere:

Use this specifically to get Player Names or Upcoming Fixtures (dates of games not yet played). This ticks the box: "Used a Web API" (1)

In [3]:
import requests
import pandas as pd

# 1. The Official FPL Fixtures API Endpoint
url = "https://fantasy.premierleague.com/api/fixtures/"

print("Fetching data from FPL API...")
response = requests.get(url)

if response.status_code == 200:
    # 2. The API returns JSON
    data = response.json()
    all_matches = pd.DataFrame(data)
    
    # 3. Filter for matches that are 'finished' and unfinished or upcoming
    completed_matches = all_matches[all_matches['finished'] == True].copy()
    upcoming_matches = all_matches[all_matches['finished'] == False].copy()

    # 4. Select and Rename relevant columns
    # team_h = Home Team ID, team_a = Away Team ID
    # team_h_score = Home Goals, team_a_score = Away Goals
    columns_to_keep = ['event', 'kickoff_time', 'team_h', 'team_a', 'team_h_score', 'team_a_score']
    
    df_2025_completed = completed_matches[columns_to_keep].copy()
    df_2025_upcoming = upcoming_matches[columns_to_keep].copy()
    
    # 5. Map Team IDs to Names (The API uses IDs like 11 for Liverpool)
    # We fetch the team list to map IDs to Names
    bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    teams_df = pd.DataFrame(requests.get(bootstrap_url).json()['teams'])
    id_to_name = dict(zip(teams_df['id'], teams_df['name']))
    
    # Apply the mapping
    df_2025_completed['Home_Team'] = df_2025_completed['team_h'].map(id_to_name)
    df_2025_completed['Away_Team'] = df_2025_completed['team_a'].map(id_to_name)

    df_2025_upcoming['Home_Team'] = df_2025_upcoming['team_h'].map(id_to_name)
    df_2025_upcoming['Away_Team'] = df_2025_upcoming['team_a'].map(id_to_name)
    
    # Clean up the final dataframe
    final_df_2025_completed = df_2025_completed[['kickoff_time', 'Home_Team', 'Away_Team', 'team_h_score', 'team_a_score']]
    final_df_2025_completed.columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'] # Standard names
    print(f"Success! Retrieved {len(final_df_2025_completed)} finished matches from the 2025/26 season.")
    display(final_df_2025_completed.head())

    final_df_2025_upcoming = df_2025_upcoming[['kickoff_time', 'Home_Team', 'Away_Team', 'team_h_score', 'team_a_score']]
    final_df_2025_upcoming.columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'] # Standard names    
    print(f"Success! Retrieved {len(final_df_2025_upcoming)} upcoming matches from the 2025/26 season.")
    display(final_df_2025_upcoming.head())

else:
    print("API Failed.")

Fetching data from FPL API...
Success! Retrieved 130 finished matches from the 2025/26 season.


,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2025-08-15T19:00:00Z,Liverpool,Bournemouth,4.0,2.0
1,2025-08-16T11:30:00Z,Aston Villa,Newcastle,0.0,0.0
2,2025-08-16T14:00:00Z,Brighton,Fulham,1.0,1.0
3,2025-08-16T14:00:00Z,Spurs,Burnley,3.0,0.0
4,2025-08-16T14:00:00Z,Sunderland,West Ham,3.0,0.0


Success! Retrieved 250 upcoming matches from the 2025/26 season.


,Date,HomeTeam,AwayTeam,FTHG,FTAG
130,2025-12-02T19:30:00Z,Bournemouth,Everton,NaN,NaN
131,2025-12-02T19:30:00Z,Fulham,Man City,NaN,NaN
132,2025-12-02T20:15:00Z,Newcastle,Spurs,NaN,NaN
133,2025-12-03T19:30:00Z,Arsenal,Brentford,NaN,NaN
134,2025-12-03T19:30:00Z,Brighton,Aston Villa,NaN,NaN


get more enriched data (relevant for analysis):

In [4]:
import pandas as pd

# The URL pattern is always 'mmz4281' + 'Season(2526)' + 'LeagueCode(E0)'
# E0 = English Premier League
url_rich_data = "https://www.football-data.co.uk/mmz4281/2526/E0.csv"

# Load directly into pandas
df_rich = pd.read_csv(url_rich_data)

# Let's see what columns we have now
print("Columns available:", df_rich.columns.tolist())
display(df_rich.head())

Columns available: ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BFDH', 'BFDD', 'BFDA', 'BMGMH', 'BMGMD', 'BMGMA', 'BVH', 'BVD', 'BVA', 'BWH', 'BWD', 'BWA', 'CLH', 'CLD', 'CLA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BFEH', 'BFED', 'BFEA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'BFE>2.5', 'BFE<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'BFEAHH', 'BFEAHA', 'B365CH', 'B365CD', 'B365CA', 'BFDCH', 'BFDCD', 'BFDCA', 'BMGMCH', 'BMGMCD', 'BMGMCA', 'BVCH', 'BVCD', 'BVCA', 'BWCH', 'BWCD', 'BWCA', 'CLCH', 'CLCD', 'CLCA', 'LBCH', 'LBCD', 'LBCA', 'PSCH', 'PSCD', 'PSCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,15/08/2025,20:00,Liverpool,Bournemouth,4,2,H,1,0,...,2.03,1.78,2.07,1.85,2.03,1.88,1.94,1.76,2.14,1.86
1,E0,16/08/2025,12:30,Aston Villa,Newcastle,0,0,D,0,0,...,2.05,1.80,2.02,1.89,2.06,1.80,1.95,1.74,2.14,1.86
2,E0,16/08/2025,15:00,Brighton,Fulham,1,1,D,0,0,...,1.83,2.03,1.93,2.00,1.84,2.03,1.80,1.96,1.91,2.08
3,E0,16/08/2025,15:00,Sunderland,West Ham,3,0,H,0,0,...,1.95,1.90,1.97,1.95,1.95,1.94,1.86,1.78,2.02,1.97
4,E0,16/08/2025,15:00,Tottenham,Burnley,3,0,H,1,0,...,1.98,1.88,1.99,1.93,1.98,1.91,1.88,1.83,2.07,1.92


## further to do:
- Data Merging: Combine the datasets (or just use the CSV for the modeling part).
- figure out what data is needed for analysis
- build database for this data
- analyze